In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile("drive/My Drive/Data4.zip","r") as zip_ref:
  zip_ref.extractall()

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing import image
import keras.utils as image
from keras import utils as np_utils
%matplotlib inline
train = pd.read_csv('drive/My Drive/Traindata18.csv')
TRAIN_PATH ='Data4/Newdata/'
from PIL import Image
from imageio import imread
from skimage.transform import resize
import cv2
from tqdm import tqdm
# defining a function to read images
def read_img(img_path):
  img = tf.keras.utils.load_img(img_path, target_size=[64,64,1],grayscale=True)
  img = img.convert(mode='RGB') #makes 3 channels
  img = image.img_to_array(img)
  img = img/255
  return img
# reading the images
train_img = []
for img_path in tqdm(train.Image.values):
  train_img.append(read_img(TRAIN_PATH + img_path))

  0%|          | 0/10772 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(
100%|██████████| 10772/10772 [00:05<00:00, 1867.59it/s]


In [ ]:
import keras
from keras import utils as np_utils
X_train = np.array(train_img)
print("Length of total input:",len(X_train))
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(train.Label.values)
Y_train = lb.transform(train.Label.values)
Y_train=Y_train.astype(np.int32)
Y_train = keras.utils.np_utils.to_categorical(Y_train)
split_size = int(X_train.shape[0]*0.8)
x_train, x_test = X_train[:split_size], X_train[split_size:]
y_train, y_test = Y_train[:split_size], Y_train[split_size:]
print(x_train.shape, x_test.shape,)
print(y_train.shape, y_test.shape)
print(len(y_test))

Length of total input: 10772
(8617, 64, 64, 3) (2155, 64, 64, 3)
(8617, 10) (2155, 10)
2155


In [ ]:
print(y_test[1:5])
image_shape = x_train.shape[1:]
n_classes = y_train.shape[1]
print("Number of training examples =", x_train.shape[0])
print("Number of testing examples =", x_test.shape[0])
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
img_rows, img_cols = 64, 64
input_shape = (img_rows, img_cols, 3)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Number of training examples = 8617
Number of testing examples = 2155
Image data shape = (64, 64, 3)
Number of classes = 10


In [ ]:
import tensorflow as tf
import keras
import keras.utils
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.core import Dense, Activation, Dropout
from keras.regularizers import l1

In [ ]:
#Build the model
from keras.applications.resnet import ResNet50

'The first base model used is VGG19. The pretrained weights from the imagenet challenge are used'
#base_model_1 = VGG19(include_top=False,weights='imagenet',input_shape=(64,64,3),classes=n_classes)

'For the 2nd base model we will use Resnet 50 and compare the performance against the previous one.The hypothesis is that Resnet 50 should perform better because of its deeper architecture'
base_model_1 = ResNet50(include_top=False,weights='imagenet',input_shape=(64,64,3),classes=n_classes)

#Lets add the final layers to these base models where the actual classification is done in the dense layers

model_1= Sequential()
model_1.add(base_model_1) #Adds the base model (in this case vgg19 to model_1)
model_1.add(Flatten()) #Since the output before the flatten layer is a matrix we have to use this function to get a vector of the form nX1 to feed it into the fully connected layers

#Add the Dense layers along with activation and batch normalization
model_1.add(Dense(1024,activation=('relu'),input_dim=512))
model_1.add(Dense(512,activation=('relu')))
model_1.add(Dense(256,activation=('relu')))
#model_1.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights

#model_1.add(Dropout(.2))

model_1.add(Dense(10,activation=('softmax'))) #This is the classification layer

#Check final model summary
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              8389632   
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 10)                2570      
                                                                 
Total params: 32,636,042
Trainable params: 32,582,922


In [ ]:
from tensorflow.keras.optimizers import Adam
model_1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adam( learning_rate=0.0001,amsgrad=False,name="Adam",),metrics=['accuracy'])
history3 = model_1.fit(x_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(x_test, y_test))

Epoch 1/100
270/270 [==============================] - 55s 73ms/step - loss: 0.2441 - accuracy: 0.9307 - val_loss: 142.1959 - val_accuracy: 0.1044
Epoch 2/100
270/270 [==============================] - 17s 64ms/step - loss: 0.0662 - accuracy: 0.9799 - val_loss: 6.5412 - val_accuracy: 0.0812
Epoch 3/100
270/270 [==============================] - 17s 63ms/step - loss: 0.0598 - accuracy: 0.9863 - val_loss: 4.6217 - val_accuracy: 0.2190
Epoch 4/100
270/270 [==============================] - 17s 62ms/step - loss: 0.0374 - accuracy: 0.9890 - val_loss: 0.5922 - val_accuracy: 0.8478
Epoch 5/100
270/270 [==============================] - 17s 63ms/step - loss: 0.0247 - accuracy: 0.9925 - val_loss: 0.0947 - val_accuracy: 0.9735
Epoch 6/100
270/270 [==============================] - 17s 64ms/step - loss: 0.0711 - accuracy: 0.9826 - val_loss: 0.0957 - val_accuracy: 0.9722
Epoch 7/100
270/270 [==============================] - 17s 62ms/step - loss: 0.0308 - accuracy: 0.9920 - val_loss: 0.0352 - val_

In [ ]:
loss, accuracy=model_1.evaluate(x_test, y_test, verbose=0)
print(accuracy*100)
print(loss)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras import utils as np_utils
%matplotlib inline
train = pd.read_csv('drive/My Drive/Traindata18.csv')
TRAIN_PATH ='Data4/Newdata/'
from PIL import Image
from imageio import imread
from skimage.transform import resize
import cv2
from tqdm import tqdm
# defining a function to read images
def read_img(img_path):
  img = image.load_img(img_path, target_size=[64,64,1],grayscale=True)
  img = img.convert(mode='RGB') #makes 3 channels
  img = image.img_to_array(img)
  img = img/255
  return img
# reading the images
train_img = []
for img_path in tqdm(train.Image.values):
  train_img.append(read_img(TRAIN_PATH + img_path))

In [ ]:
import keras
from keras import utils as np_utils
X_train = np.array(train_img)
print("Length of total input:",len(X_train))
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(train.Label.values)
Y_train = lb.transform(train.Label.values)
Y_train=Y_train.astype(np.int32)
Y_train = keras.utils.np_utils.to_categorical(Y_train)
split_size = int(X_train.shape[0]*0.8)
x_train, x_test = X_train[:split_size], X_train[split_size:]
y_train, y_test = Y_train[:split_size], Y_train[split_size:]
print(x_train.shape, x_test.shape,)
print(y_train.shape, y_test.shape)
print(len(y_test))

In [ ]:
print(y_test[1:5])
image_shape = x_train.shape[1:]
n_classes = y_train.shape[1]
print("Number of training examples =", x_train.shape[0])
print("Number of testing examples =", x_test.shape[0])
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
img_rows, img_cols = 64, 64
input_shape = (img_rows, img_cols, 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#reading dataset using pandas
data_test = pd.read_csv('/content/drive/MyDrive/TestSAR.csv')
data_train = pd.read_csv('/content/drive/MyDrive/TrainSAR.csv')


img_rows, img_cols = 64, 64
input_shape = (img_rows, img_cols)
#taking the pixel values from the training data. The pixel values start from location 1 in the csv file till the 4096
X = np.array(data_train.iloc[:, 1:])
#convert the label type to the string value
y = to_categorical(np.array(data_train.iloc[:, 0]).astype('str'))

#Here we split validation data to optimiza classifier during training
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

#Test data
X_test = np.array(data_test.iloc[:, 1:])
#one hot encoding
y_test = to_categorical(np.array(data_test.iloc[:, 0]))


#X_train.shape=(number of images in the index 0, rowsize, columnsize)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols)
X_train = np.stack((X_train,)*3,axis=-1) #makes 3 channels
X_test = np.stack((X_test,)*3,axis=-1)
X_val = np.stack((X_val,)*3,axis=-1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_test /= 255
X_val /= 255

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(22531, 64, 64, 3)
(9390, 64, 64, 3)
(5633, 64, 64, 3)


In [ ]:
#Build the model
from keras.applications.resnet import ResNet50
import keras
import keras.utils
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.core import Dense, Activation, Dropout
from keras.regularizers import l1

'The first base model used is VGG19. The pretrained weights from the imagenet challenge are used'
#base_model_1 = VGG19(include_top=False,weights='imagenet',input_shape=(64,64,3),classes=n_classes)

'For the 2nd base model we will use Resnet 50 and compare the performance against the previous one.The hypothesis is that Resnet 50 should perform better because of its deeper architecture'
base_model_1 = ResNet50(include_top=False,weights='imagenet',input_shape=(64,64,3),classes=10)

#Lets add the final layers to these base models where the actual classification is done in the dense layers

model_1= Sequential()
model_1.add(base_model_1) #Adds the base model (in this case vgg19 to model_1)
model_1.add(Flatten()) #Since the output before the flatten layer is a matrix we have to use this function to get a vector of the form nX1 to feed it into the fully connected layers

#Add the Dense layers along with activation and batch normalization
model_1.add(Dense(1024,activation=('relu'),input_dim=512))
model_1.add(Dense(512,activation=('relu')))
model_1.add(Dense(256,activation=('relu')))
#model_1.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights

#model_1.add(Dropout(.2))
model_1.add(Dense(10,activation=('softmax'))) #This is the classification layer

#Check final model summary
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 32,636,042
Trainable params: 32,582,922
No

In [ ]:
from tensorflow.keras.optimizers import Adam
model_1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adam( learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.9,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",),
               metrics=['accuracy'])
history3 = model_1.fit(X_train, y_train,
          batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test))

Epoch 1/100
705/705 [==============================] - 94s 71ms/step - loss: 0.6281 - accuracy: 0.8047 - val_loss: 31.0355 - val_accuracy: 0.0501
Epoch 2/100
705/705 [==============================] - 46s 65ms/step - loss: 0.1820 - accuracy: 0.9416 - val_loss: 0.4768 - val_accuracy: 0.8619
Epoch 3/100
705/705 [==============================] - 46s 65ms/step - loss: 0.1103 - accuracy: 0.9637 - val_loss: 0.1741 - val_accuracy: 0.9529
Epoch 4/100
705/705 [==============================] - 46s 65ms/step - loss: 0.0733 - accuracy: 0.9764 - val_loss: 0.1451 - val_accuracy: 0.9608
Epoch 5/100
705/705 [==============================] - 46s 66ms/step - loss: 0.0518 - accuracy: 0.9838 - val_loss: 0.1477 - val_accuracy: 0.9626
Epoch 6/100
705/705 [==============================] - 46s 66ms/step - loss: 0.0402 - accuracy: 0.9875 - val_loss: 0.1740 - val_accuracy: 0.9561
Epoch 7/100
169/705 [======>.......................] - ETA: 30s - loss: 0.0312 - accuracy: 0.9904

KeyboardInterrupt: ignored

In [ ]:
loss, accuracy=model_1.evaluate(x_test, y_test, verbose=0)
print(accuracy*100)
print(loss)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import os
import pickle
model_1.save('Model_Texture_Resnet.h5')
save_history(history3, 'hist1.bin')
history=load_history('hist1.bin')
plot_compare(history)